# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig



## Dataset

### Overview
The dataset to be used is built up from meteorological values for the city of Los Angeles, USA. It includes data like the minimum and maximum temperature on the day, the wind speed and the measured precipitation. 
The objective of this project will be to make a model capable of predicting the precipitation in the city depending on the other given meteorogical measures. For this we will use the autoML tool from Azure as well as a predifined regression model with optimized hyperparameters using HyperDrive.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Capstone_ML'

experiment=Experiment(ws, experiment_name)

found = False
key = "Precipitation-LA"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        

if not found:
        # Gets a dataset from a given URL
        example_data = 'https://raw.githubusercontent.com/RaikohGrass/ML_Azure_Capstone/main/precipitation_LA.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        
        #Register the dataset in the default datastore
        dataset = dataset.register(workspace=ws,
                                   name=key)


df = dataset.to_pandas_dataframe()
df.describe()


,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
count,1823.000000,1827.000000,1827.000000,1827.000000,1822.000000,1809.000000,1823.000000,1809.000000,551.0,39.0,861.0
mean,1.675710,0.035720,76.498632,58.120416,254.143798,256.428966,9.062095,14.721669,1.0,1.0,1.0
std,0.909434,0.189859,9.133929,7.223756,54.756284,55.571902,2.016827,3.622257,0.0,0.0,0.0
min,0.000000,0.000000,52.000000,38.000000,10.000000,10.000000,2.900000,6.000000,1.0,1.0,1.0
25%,1.120000,0.000000,70.000000,53.000000,260.000000,270.000000,8.100000,12.100000,1.0,1.0,1.0
50%,1.570000,0.000000,76.000000,58.000000,270.000000,270.000000,8.900000,14.100000,1.0,1.0,1.0
75%,2.010000,0.000000,83.000000,64.000000,270.000000,280.000000,10.100000,16.100000,1.0,1.0,1.0
max,8.050000,2.670000,111.000000,79.000000,360.000000,360.000000,21.000000,34.000000,1.0,1.0,1.0


In [6]:
# We drop the TAVG column, as it is empty
dataset2 = dataset.drop_columns('TAVG')
df =dataset2.to_pandas_dataframe()
df.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
0,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-01,2.46,None,0.00,64,43,10.0,30.0,8.1,11.0,NaN,NaN,1.0
1,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-02,2.01,None,0.00,65,47,270.0,30.0,6.0,8.9,NaN,NaN,NaN
2,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-03,0.67,None,0.00,62,44,150.0,150.0,10.1,14.1,NaN,NaN,NaN
3,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-04,1.34,None,0.01,69,55,270.0,280.0,8.1,14.1,NaN,NaN,NaN
4,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-05,2.46,None,1.61,59,49,140.0,140.0,10.1,16.1,1.0,1.0,NaN


In [33]:
from sklearn import model_selection
X = df.drop(columns=['PRCP'])
y = df['PRCP']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

### Creating the compute target
Here we will define the compute cluster for our runs

In [7]:
cpu_cluster_name = 'EduardoCluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'normalized_root_mean_squared_error'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target='EduardoCluster',
                             task = "regression",
                             training_data=dataset2,
                             label_column_name="PRCP", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Capstone_ML,AutoML_4a965d05-caf2-4a03-961a-6e126549f924,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

We can retrieve the best model from the autoML run



In [11]:
best_autoML_run, best_autmoML_model = remote_run.get_output()

From the properties we can see that the best model was a Voting Ensemble using the following algorithms:

- 'ExtremeRandomTrees', 
-  4 'XGBoostRegressor'
- 'DecisionTree'

In [13]:
best_autoML_run.get_metrics()

{'median_absolute_error': 0.002094368448689829,
 'explained_variance': 0.48617888222545896,
 'normalized_root_mean_squared_error': 0.05010104750828163,
 'spearman_correlation': 0.37908443759782123,
 'normalized_mean_absolute_error': 0.014245698949873986,
 'r2_score': 0.48603031262802904,
 'normalized_median_absolute_error': 0.000784407658685329,
 'root_mean_squared_log_error': nan,
 'root_mean_squared_error': 0.13376979684711196,
 'mean_absolute_error': 0.03803601619616354,
 'normalized_root_mean_squared_log_error': nan,
 'mean_absolute_percentage_error': 148.62075874523455,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_4a965d05-caf2-4a03-961a-6e126549f924_38/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_4a965d05-caf2-4a03-961a-6e126549f924_38/predicted_true'}

In [12]:
best_autoML_run.get_details()

{'runId': 'AutoML_4a965d05-caf2-4a03-961a-6e126549f924_38',
 'target': 'EduardoCluster',
 'status': 'Completed',
 'startTimeUtc': '2022-01-26T13:46:47.414372Z',
 'endTimeUtc': '2022-01-26T13:47:47.915268Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'regression\',\'primary_metric\':\'normalized_root_mean_squared_error\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'Capstone_ML\',\'compute_target\':\'EduardoCluster\',\'subscription_id\':\'f9d5a085-54dc-4215-9ba6-dad5d86e60a0\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_4a965d05-caf2-4a03-961a-6e126549f924_38","experiment_name":"Capstone_ML","workspace_name":"quick-starts-ws-178857","subs

We then proceed to register the model for later deployment

In [21]:
#TODO: Save the best model
autoML_model = best_autoML_run.register_model(model_name = 'best_autoML_model', model_path = './outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
from azureml.core import Model
from azureml.core.conda_dependencies import CondaDependencies

environment = best_autoML_run.get_environment()

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


# we retrieve the score script generated with the registered model
save_path = 'score.py'
best_autoML_run.download_file('outputs/scoring_file_v_1_0_0.py', save_path)

inference_config = InferenceConfig(entry_script = save_path, environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2,description = 'precipitation predicter for LA')

service_name = 'precipitation-predictor'
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[autoML_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)


In [24]:
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-26 14:39:23+00:00 Creating Container Registry if not exists.
2022-01-26 14:39:24+00:00 Registering the environment.
2022-01-26 14:39:24+00:00 Use the existing image.
2022-01-26 14:39:24+00:00 Generating deployment configuration.
2022-01-26 14:39:25+00:00 Submitting deployment to compute.
2022-01-26 14:39:30+00:00 Checking the status of deployment precipitation-predictor..
2022-01-26 14:40:06+00:00 Checking the status of inference endpoint precipitation-predictor.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [30]:
print('Service REST URI: {}'.format(service.scoring_uri))
print('Swagger URI: {}'.format(service.swagger_uri))

Service REST URI: http://3e08f3a6-d113-4df7-a407-8fc9518f8ba4.southcentralus.azurecontainer.io/score
Swagger URI: http://3e08f3a6-d113-4df7-a407-8fc9518f8ba4.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
